In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import boxcox
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

client=pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/client.csv")
elec=pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv")
fore=pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv")
gas=pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv")
hist=pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv")
train = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/train.csv')
location=pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv')
location.dropna(inplace=True)
location.drop(columns='county_name',inplace=True)
def preprocess():
 
    train['datetime']=pd.to_datetime(train['datetime'])
    elec['forecast_date']=pd.to_datetime(elec['forecast_date'])
    elec.rename(columns={'forecast_date':'datetime'},inplace=True)

    
    
    fore['forecast_datetime']=pd.to_datetime(fore['forecast_datetime'])
    fore.rename(columns={'forecast_datetime':'datetime'},inplace=True)



    fore['latitude']=fore['latitude'].astype('float').round(1)
    fore['longitude']=fore['longitude'].astype('float').round(1)
    fore_location=pd.merge(fore,location,on=['latitude','longitude'],how='left').dropna(axis=0).drop(columns=['hours_ahead','origin_datetime','latitude','longitude','data_block_id'])
    fore_location['county']=fore_location['county'].astype('int')
    
    for cols in fore_location.columns:      
        if(cols!='county' and cols!='datetime' and cols!='data_block_id'):
            new_name=f'forecast_{cols}'
            fore_location.rename(columns={cols:new_name},inplace=True)    
    
    fore_location['datetime']=pd.to_datetime(fore_location['datetime'])
    
    fore_location_overall=fore_location.groupby(by=['datetime']).mean().drop(columns=['county']).reset_index()

    fore_location_overall['datetime']=pd.to_datetime(fore_location_overall['datetime'])

    fore_location_county=fore_location.groupby(by=['datetime','county']).mean().reset_index()

    fore_location_county['datetime']=pd.to_datetime(fore_location_overall['datetime'])
    
    
    for cols in fore_location_overall.columns:
        if(cols!='datetime' and cols!='data_block_id' and cols!='county'):
            new_name=f'overall_{cols}'
            fore_location_overall.rename(columns={cols:new_name},inplace=True)
    for cols in fore_location_county.columns:
        if(cols!='datetime' and cols!='county' and cols!='data_block_id'):
            new_name=f'local_{cols}'
            fore_location_county.rename(columns={cols:new_name},inplace=True)


            
    hist['latitude']=hist['latitude'].astype('float').round(1)
    hist['longitude']=hist['longitude'].astype('float').round(1)
    location_df_history=pd.merge(hist,location,on=['latitude','longitude'],how='left').dropna().drop(columns=['latitude','longitude'])

    for cols in location_df_history.columns:
        if(cols!='county' and cols!='datetime' and cols!='data_block_id'):
            new_name=f'historic_{cols}'
            location_df_history.rename(columns={cols:new_name},inplace=True)
    location_df_history['county']=location_df_history['county'].astype('int')
    location_df_history['datetime']=pd.to_datetime(location_df_history['datetime'])
    
    df_overall_historic=location_df_history.groupby(by=['datetime']).mean().reset_index().drop(columns=['county','data_block_id'])
    df_overall_historic['datetime']=pd.to_datetime(df_overall_historic['datetime'])
    df_overall_historic=pd.merge(df_overall_historic,location_df_history[['datetime', 'data_block_id']],on=['datetime'],how='left')
    
    df_local_historic=location_df_history.groupby(by=['datetime','county']).mean().reset_index().drop(columns=['data_block_id'])
    df_local_historic['datetime']=pd.to_datetime(df_local_historic['datetime'])
    df_local_historic=pd.merge(df_local_historic,location_df_history[['datetime', 'data_block_id']],on=['datetime'],how='left')
    
    for cols in df_overall_historic.columns:
        if(cols!='datetime' and cols!='data_block_id' and cols!='county'):
            new_name=f'overall_{cols}'
            df_overall_historic.rename(columns={cols:new_name},inplace=True)
    for cols in df_local_historic.columns:
        if(cols!='datetime' and cols!='county' and cols!='data_block_id'):
            new_name=f'local_{cols}'
            df_local_historic.rename(columns={cols:new_name},inplace=True)
            
            
    train['year']=train['datetime'].dt.year
    train['month']=train['datetime'].dt.month
    train['day']=train['datetime'].dt.day
    train['hour']=train['datetime'].dt.hour            
    
    
    elec['hour'] = elec['datetime'].dt.hour
    
    
    
#     merged_data= pd.merge(train,client.drop(columns = ['date']), how='left', on=['data_id', 'county', 'is_business', 'product_type'])
    merged_data= pd.merge(train,gas[['data_block_id', 'lowest_price_per_mwh', 'highest_price_per_mwh']], how='left', on='data_block_id')
    merged_data= pd.merge(merged_data,client.drop(columns = ['date']), how='left', on=['data_block_id', 'county', 'is_business', 'product_type'])
    merged_data= pd.merge(merged_data,elec[['euros_per_mwh', 'hour', 'data_block_id']], how='left', on=['hour', 'data_block_id'])
    merged_data= pd.merge(merged_data,fore_location_overall, how='left', on=['datetime'])
    merged_data= pd.merge(merged_data,fore_location_county, how='left', on=['datetime', 'county'])
    
    
    
    
    df_overall_historic['hour']= df_overall_historic['datetime'].dt.hour
    df_local_historic['hour']= df_local_historic['datetime'].dt.hour

    df_overall_historic.drop_duplicates(inplace=True)
    df_local_historic.drop_duplicates(inplace=True)
    df_overall_historic.drop('datetime', axis= 1, inplace= True)
    df_local_historic.drop('datetime', axis= 1, inplace= True)

    
    merged_data= merged_data.merge(df_overall_historic, how='left', on=['data_block_id', 'hour'])

    merged_data= merged_data.merge(df_local_historic, how='left', on=['data_block_id', 'county', 'hour'])
   
    merged_data=merged_data.groupby(['year', 'day', 'hour'], as_index=False).apply(lambda x: x.ffill().bfill()).reset_index()

    merged_data.drop(['level_0', 'level_1', 'row_id', 'data_block_id'], axis= 1, inplace= True)
    return merged_data

In [ ]:
full_merged_df=preprocess()

In [ ]:
# full_merged_df.to_csv('enfit.csv',index=False)

In [ ]:
full_merged_df.columns

In [ ]:
def generate_features(df):
    df["dayofyear"] = df["datetime"].dt.dayofyear
    df["sin(dayofyear)"] = np.sin(np.pi * df["dayofyear"] / 183)
    df["cos(dayofyear)"] = np.cos(np.pi * df["dayofyear"] / 183)
    df["sin(hour)"] = np.sin(np.pi * df["hour"] / 12)
    df["cos(hour)"] = np.cos(np.pi * df["hour"] / 12)
    return df

In [ ]:
full_merged_df=generate_features(full_merged_df)

In [ ]:
full_merged_df

In [ ]:
def cubic_root_transform(series):
    return np.cbrt(series)

def cubic_root_transform_column(series):
    return np.cbrt(series)

def cubic_root_transform_df(df, cols):
    for col in cols:
        if df[col].skew() >= 0.1:
            print('Right-skewed col:', col)
            df[col] = cubic_root_transform_column(df[col])
        elif df[col].skew() <= -0.1:
            df[col] = np.clip(df[col], df[col].quantile(0.01), df[col].quantile(0.99))
            print('Left-skewed col:', col)
            df[col] = cubic_root_transform_column(df[col])
    return df

dropcols = ['county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'prediction_unit_id', 'year', 'month', 'day', 'hour','sin(dayofyear)','cos(dayofyear)','sin(hour)','cos(hour)']
transform_cols = list(full_merged_df.columns)
for cols in dropcols:
    transform_cols.remove(cols)

full_merged_df = cubic_root_transform_df(full_merged_df, transform_cols)

# apply standard scalar:
transform_cols.remove('target')
columns_to_scale=transform_cols
feature_scaler = StandardScaler()
# target_scaler = StandardScaler()
full_merged_df[columns_to_scale] = feature_scaler.fit_transform(full_merged_df[columns_to_scale])
# full_merged_df['target'] = target_scaler.fit_transform(full_merged_df[['target']])

In [ ]:
temp_df=full_merged_df[['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'prediction_unit_id', 'year', 'month', 'day', 'hour',
       'lowest_price_per_mwh', 'highest_price_per_mwh', 'eic_count',
       'installed_capacity', 'euros_per_mwh']]
correlation_matrix = temp_df.corr()
import seaborn as sns
plt.figure(figsize=(8, 6))  # Specify the figure size
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')  # Add a title
plt.show()

In [ ]:
temp_df=full_merged_df[['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'prediction_unit_id', 'year', 'month', 'day', 'hour',
       'lowest_price_per_mwh', 'highest_price_per_mwh', 'eic_count',
       'installed_capacity', 'euros_per_mwh']]
exclude_cols=['is_business', 'product_type','is_consumption','eic_count','installed_capacity','target','datetime']
from sklearn.decomposition import PCA
pca = PCA(n_components=5)  # Reduce to 2 dimensions
y_df=temp_df[exclude_cols]
df_pca = pca.fit_transform(temp_df.drop(columns=exclude_cols))
df_pca=pd.DataFrame(df_pca)
df_pca[exclude_cols]=y_df[exclude_cols]
correlation_matrix = df_pca.corr()
plt.figure(figsize=(8, 6))  # Specify the figure size
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')  # Add a title
plt.show()

In [ ]:
full_merged_df.drop(columns=['datetime'],inplace=True)

In [ ]:
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
X = full_merged_df.drop(columns=['target'])
y = full_merged_df['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the regressors
xgb_regressor = XGBRegressor(n_estimators=600,
    learning_rate=0.02,
    max_depth= 5,
    subsample= 0.8,
    colsample_bytree=0.9)
lgbm_regressor = LGBMRegressor(
    n_estimators= 500,
    learning_rate= 0.1,
    max_depth= 6,
    subsample=0.9,
    colsample_bytree= 0.8)
catboost_regressor = CatBoostRegressor(
    iterations= 300,
    learning_rate= 0.1,
    depth= 5,
    subsample=0.9)

# Create the ensemble
ensemble_regressor = VotingRegressor(estimators=[
    ('xgb', xgb_regressor),
    ('lgbm', lgbm_regressor),
    ('catboost', catboost_regressor)
])

# Fit the ensemble on the training data
ensemble_regressor.fit(X_train, y_train)

# Make predictions on the test data
ensemble_preds = ensemble_regressor.predict(X_test)

# Evaluate the performance
rmse = mean_squared_error(y_test, ensemble_preds, squared=False)
print(f'Ensemble RMSE: {rmse}')
# Calculate the R^2 score
r2 = r2_score(y_test, ensemble_preds)
print(f'Ensemble R^2 Score: {r2}')

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#     full_merged_df.drop(columns='target'),
#     full_merged_df['target'],
#     test_size=0.33,
#     random_state=42
# )
# p_1={'n_iter':4000,'verbose': -1,'random_state':73,'objective':'tweedie','learning_rate': 0.004811751415536496, 'colsample_bytree': 0.8841841689852410, 'colsample_bynode': 0.4305836942635745, 'reg_alpha': 3.11984157361821, 'reg_lambda': 1.088469732297296, 'min_data_in_leaf': 162, 'max_depth': 16,'device':'gpu' , 'num_leaves': 435,'n_jobs':4}
# p0 ={'n_iter':4000,'verbose': -1,'random_state':73,'objective':'tweedie','learning_rate': 0.007922862526647507, 'colsample_bytree': 0.9052952790963521, 'colsample_bynode': 0.4416947152746856, 'reg_alpha': 3.31471952672932, 'reg_lambda': 1.349570843308307, 'min_data_in_leaf': 185, 'max_depth': 18,'device':'gpu' , 'num_leaves': 445,'n_jobs':4}
# p1 ={'n_iter':4000,'verbose': -1,'random_state':73,'objective':'tweedie','learning_rate': 0.010339736147758608, 'colsample_bytree': 0.9263063801074632, 'colsample_bynode': 0.4527058263857967, 'reg_alpha': 3.62802063709343, 'reg_lambda': 1.650681954419419, 'min_data_in_leaf': 201, 'max_depth': 20,'device':'gpu' , 'num_leaves': 455,'n_jobs':4}
# p2 ={'n_iter':4000,'verbose': -1,'random_state':73,'objective':'tweedie','learning_rate': 0.013090718804096083, 'colsample_bytree': 0.9499770953943448, 'colsample_bynode': 0.4670163857441046, 'reg_aplha': 3.96946065556807, 'reg_lambda': 1.925712107567988, 'min_data_in_leaf': 223, 'max_depth': 22,'device':'gpu' , 'num_leaves': 465,'n_jobs':4}
# p3 ={'n_iter':4000,'verbose': -1,'random_state':73,'objective':'tweedie','learning_rate': 0.015559490612977255, 'colsample_bytree': 0.9682791614810814, 'colsample_bynode': 0.4722023075509447, 'reg_aplha': 4.15624585398345, 'reg_lambda': 2.265053303366992, 'min_data_in_leaf': 254, 'max_depth': 24,'device':'gpu' , 'num_leaves': 475,'n_jobs':4}
# p4 ={'n_iter':4000,'verbose': -1,'random_state':73,'objective':'tweedie','learning_rate': 0.018908744594789185, 'colsample_bytree': 0.9864875442500248, 'colsample_bynode': 0.4832525869590394, 'reg_aplha': 4.35845913192557, 'reg_lambda': 2.355521088983217, 'min_data_in_leaf': 289, 'max_depth': 26,'device':'gpu' , 'num_leaves': 485,'n_jobs':4}
# p5 ={'n_iter':4000,'verbose': -1,'random_state':73,'objective':'tweedie','learning_rate': 0.021819855605890296, 'colsample_bytree': 0.9995986553611359, 'colsample_bynode': 0.4953634970601405, 'reg_alpha': 4.58956024201648, 'reg_lambda': 2.616432197094328, 'min_data_in_leaf': 309, 'max_depth': 28,'device':'gpu' , 'num_leaves': 495,'n_jobs':4}
# lgbp_1=LGBMRegressor(**p_1)
# lgbp0=LGBMRegressor(**p0)
# lgbp1=LGBMRegressor(**p1)
# lgbp2=LGBMRegressor(**p2)
# lgbp3=LGBMRegressor(**p3)
# lgbp4=LGBMRegressor(**p4)
# lgbp5=LGBMRegressor(**p5)

# for lgbm_model in [lgbp_1, lgbp0, lgbp1, lgbp2, lgbp3, lgbp4, lgbp5]: # v1
# # for lgbm_model in [lgbp_1, lgbp1, lgbp2, lgbp3, lgbp4]:
#     print('_______________________________________________________')
#     print('Start')
#     lgbm_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], callbacks=[
#             lgb.callback.early_stopping(stopping_rounds=100),
#             lgb.callback.log_evaluation(period=100),
#         ],)

In [ ]:
# from sklearn.preprocessing import normalize
# import enefit
# import pandas as pd
# import numpy as np
# env = enefit.make_env()
# iter_test = env.iter_test()
# for (train, t_re, client, hist,
#     fore, elec, gas, sample_prediction) in iter_test:
#     gas = gas.tail(1)
#     train['data_id']=0
#     client['data_id']=0
#     hist['data_id']=0
#     fore['data_id']=0
#     elec['data_id']=0
#     gas['data_id']=0  
    
#     train['prediction_datetime']=pd.to_datetime(train['prediction_datetime'])

    
    
#     train.rename(columns={'prediction_datetime':'datetime'},inplace=True)
    
#     elec['forecast_date']=pd.to_datetime(elec['forecast_date'])
#     elec.rename(columns={'forecast_date':'datetime'},inplace=True)

    
    
#     fore['forecast_datetime']=pd.to_datetime(fore['forecast_datetime'])
#     fore.rename(columns={'forecast_datetime':'datetime'},inplace=True)



#     fore['latitude']=fore['latitude'].astype('float').round(1)
#     fore['longitude']=fore['longitude'].astype('float').round(1)
#     fore_location=pd.merge(fore,location,on=['latitude','longitude'],how='left').dropna(axis=0).drop(columns=['hours_ahead','origin_datetime','latitude','longitude','data_id'])
#     fore_location['county']=fore_location['county'].astype('int')
    
    
    
#     for cols in fore_location.columns:      
#         if(cols!='county' and cols!='datetime' and cols!='data_id'):
#             new_name=f'forecast_{cols}'
#             fore_location.rename(columns={cols:new_name},inplace=True)    
    
#     fore_location['datetime']=pd.to_datetime(fore_location['datetime'])
    
#     fore_location_overall=fore_location.groupby(by=['datetime']).mean().drop(columns=['county']).reset_index()

#     fore_location_overall['datetime']=pd.to_datetime(fore_location_overall['datetime'])


    
#     fore_location_county=fore_location.groupby(by=['datetime','county']).mean().reset_index()

#     fore_location_county['datetime']=pd.to_datetime(fore_location_overall['datetime'])
    
    
#     for cols in fore_location_overall.columns:
#         if(cols!='datetime' and cols!='data_id' and cols!='county'):
#             new_name=f'overall_{cols}'
#             fore_location_overall.rename(columns={cols:new_name},inplace=True)
#     for cols in fore_location_county.columns:
#         if(cols!='datetime' and cols!='county' and cols!='data_id'):
#             new_name=f'local_{cols}'
#             fore_location_county.rename(columns={cols:new_name},inplace=True)


            
#     hist['latitude']=hist['latitude'].astype('float').round(1)
#     hist['longitude']=hist['longitude'].astype('float').round(1)
#     location_df_history=pd.merge(hist,location,on=['latitude','longitude'],how='left').dropna().drop(columns=['latitude','longitude'])

#     for cols in location_df_history.columns:
#         if(cols!='county' and cols!='datetime' and cols!='data_id'):
#             new_name=f'historic_{cols}'
#             location_df_history.rename(columns={cols:new_name},inplace=True)
#     location_df_history['county']=location_df_history['county'].astype('int')
#     location_df_history['datetime']=pd.to_datetime(location_df_history['datetime'])
    
#     df_overall_historic=location_df_history.groupby(by=['datetime']).mean().reset_index().drop(columns=['county','data_id'])
#     df_overall_historic['datetime']=pd.to_datetime(df_overall_historic['datetime'])
#     df_overall_historic=pd.merge(df_overall_historic,location_df_history[['datetime', 'data_id']],on=['datetime'],how='left')
    
#     df_local_historic=location_df_history.groupby(by=['datetime','county']).mean().reset_index().drop(columns=['data_id'])
#     df_local_historic['datetime']=pd.to_datetime(df_local_historic['datetime'])
#     df_local_historic=pd.merge(df_local_historic,location_df_history[['datetime', 'data_id']],on=['datetime'],how='left')
    
#     for cols in df_overall_historic.columns:
#         if(cols!='datetime' and cols!='data_id' and cols!='county'):
#             new_name=f'overall_{cols}'
#             df_overall_historic.rename(columns={cols:new_name},inplace=True)
#     for cols in df_local_historic.columns:
#         if(cols!='datetime' and cols!='county' and cols!='data_id'):
#             new_name=f'local_{cols}'
#             df_local_historic.rename(columns={cols:new_name},inplace=True)
            
            
#     train['year']=train['datetime'].dt.year
#     train['month']=train['datetime'].dt.month
#     train['day']=train['datetime'].dt.day
#     train['hour']=train['datetime'].dt.hour            
    
    
#     elec['hour'] = elec['datetime'].dt.hour
    
    
    
# #     merged_data= pd.merge(train,client.drop(columns = ['date']), how='left', on=['data_id', 'county', 'is_business', 'product_type'])
#     merged_data= pd.merge(train,gas[['data_id', 'lowest_price_per_mwh', 'highest_price_per_mwh']], how='left', on='data_id')
#     merged_data= pd.merge(merged_data,client.drop(columns = ['date']), how='left', on=['data_id', 'county', 'is_business', 'product_type'])
#     merged_data= pd.merge(merged_data,elec[['euros_per_mwh', 'hour', 'data_id']], how='left', on=['hour', 'data_id'])
#     merged_data= pd.merge(merged_data,fore_location_overall, how='left', on=['datetime'])
#     merged_data= pd.merge(merged_data,fore_location_county, how='left', on=['datetime', 'county'])
    
    
    
    
#     df_overall_historic['hour']= df_overall_historic['datetime'].dt.hour
#     df_local_historic['hour']= df_local_historic['datetime'].dt.hour

#     df_overall_historic.drop_duplicates(inplace=True)
#     df_local_historic.drop_duplicates(inplace=True)
#     df_overall_historic.drop('datetime', axis= 1, inplace= True)
#     df_local_historic.drop('datetime', axis= 1, inplace= True)

    
#     merged_data= merged_data.merge(df_overall_historic, how='left', on=['data_id', 'hour'])

#     merged_data= merged_data.merge(df_local_historic, how='left', on=['data_id', 'county', 'hour'])
   
#     merged_data=merged_data.groupby(['year', 'day', 'hour'], as_index=False).apply(lambda x: x.ffill().bfill()).reset_index()

#     merged_data.drop(['level_0', 'level_1', 'row_id', 'data_id','currently_scored'], axis= 1, inplace= True)
#     merged_data=generate_features(merged_data)
#     merged_data.drop(columns='datetime')
#     test_merge = cubic_root_transform_df(merged_data, transform_cols)
#     test_merge[columns_to_scale] = feature_scaler.transform(test_merge[columns_to_scale])
    
#     y_preds=ensemble_regressor.predict(test_merge)
# #     inverse_scaled_values = target_scaler.inverse_transform([y_preds])
# #     gc.collect()
    
# #     sample_prediction["target"] = inverse_scaled_values.reshape(-1,1)
#     sample_prediction["target"]=y_preds**3
#     env.predict(sample_prediction)